<a href="https://colab.research.google.com/github/probabll/demo-mbr-nmt/blob/main/German-English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intuition

# Theory

# Setting Up

In [1]:
!git clone https://github.com/probabll/demo-mbr-nmt.git

fatal: destination path 'demo-mbr-nmt' already exists and is not an empty directory.


In [2]:
cd demo-mbr-nmt

/content/demo-mbr-nmt


In [17]:
!git pull

remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 9 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), done.
From https://github.com/probabll/demo-mbr-nmt
   b09d39e..d68000f  main       -> origin/main
Updating b09d39e..d68000f
Fast-forward
 de-en.sh | 15 +++++++++------
 demo.py  | 44 +++++++++++++++++++-------------------------
 en-de.sh | 15 +++++++++------
 3 files changed, 37 insertions(+), 37 deletions(-)


In [3]:
!pip install -r requirements.txt

In [18]:
!bash de-en.sh

# Demo

In [19]:
from demo import SampleFromNMT, CachedSampler, Utility, MBR, load

The next line will take a moment.

In [20]:
model = load('de', 'en', np_seed=10, torch_seed=10)

In [24]:
example_x = "Die historischen Landkarten des digitalen Bayern-Atlases, ein Angebot des Geoportals Bayern der Staatsregierung, sind alles andere als aktuell - doch gerade deshalb sehr aufschlussreich."
example_y = "The historical maps of the digital BayernAtlas, an offering from the State Government's Geoportal Bayern, are anything but up-to-date – and yet it is precisely for this reason that they are so informative."

In [25]:
model.x.decode(model.x.encode(example_x))

'Das kennt jeder, der sich schon mal aufregen musste, weil das Auto-Navi statt einer Umgehungsstraße eine grüne Wiese anzeigte.'

In [26]:
model.y.decode(model.y.encode(example_y))

"Anyone who has ever got worked up because the car's sat-nav is showing a green field instead of a bypass knows that."

## Approximate MAP decoding

In [29]:
beam = model.x2y.beam_search(example_x, beam_size=30)
for y, surp in zip(beam['output'], beam['surprisal']):
    print(f"{surp:.4f}\t{y}")
y_map = beam['output'][0]

13.3244	This is something everyone who had to get excited about because the Auto-Navi displayed a green meadow instead of a bypass road.
13.3439	This is something everyone who had to get excited about because the Auto-Navi displayed a green meadow instead of a bypass road.
13.3562	This is something everyone who had to get excited about because the Auto-Navi showed a green meadow instead of a bypass road.
13.5292	This is something everyone who had to get excited about because the Auto-Navi showed a green meadow instead of a bypass road.
14.1213	This is something everyone who had to get excited about because the Auto @-@Navi displayed a green meadow instead of a bypass road.
13.7534	This is something everyone who had to get excited about because the AutoNavi displayed a green meadow instead of a bypass road.
13.9665	This is something everyone who had to get excited about because the AutoNavi showed a green meadow instead of a bypass road.
13.5885	This is something everyone who had to get

## Approximate MBR decoding

In [30]:
hyp_space = list(set(model.x2y.ancestral_sampling(example_x, num_samples=100)['output']))
len(hyp_space)

100

In [31]:
from sacrebleu import sentence_chrf

In [32]:
class ChrF(Utility):
    
    def __call__(self, src: str, hyp: str, ref: str) -> float:
        return sentence_chrf(hyp, [ref]).score  # note that chrf does not make use of the source sentence

chrf = ChrF()    

In [33]:
chrf('This is cool!', 'Das ist cool!', 'Cool!')

0.40623147802266224

In [34]:
chrf.batch('This is cool!', 'Das ist cool!', ['Cool!', 'Das ist cool!'])

array([0.40623148, 1.        ])

In [35]:
chrf(example_x, y_map, example_y)

0.43730321818330387

In [36]:
nmt_sampler = SampleFromNMT(model.x2y, sample_size=100)

In [37]:
nmt_sampler(example_x)

["Everyone who had to get in touch was aware of this, as the Auto-Navi didn't display a green meadow instead of a bypass.",
 'Everyone knows it because AutoNavi showed a green grass instead of a bypass road.',
 'Those who had to get excited because Auto-Navi displayed a green meadow instead of having a bypass.',
 'This became clear to everybody who had to get up late because the AutoNavi showed a green meadow instead of a bypass road.',
 'This is something everybody who had to get excited about because the Auto-Navi showed a green theme instead of a road bypass.',
 'This is what everyone who had to get up to because Auto-Navi instead of a bypass showed a meadow.',
 'Everybody who has had to get excited occasionally knows that because Auto @-@Navi displayed a green meadow instead of a bypass road.',
 "That's what anyone has been aware of, who has had to excite himself because the Auto Navi has shown a green grass instead of a bypass road.",
 'It was known by those who had to get excited

In [38]:
cached_sampler = CachedSampler(nmt_sampler, sample_size=30)

In [39]:
cached_sampler(example_x)

['So everybody knows, that simply got excited because the Auto-Navi showed a green grass in place of a bypass.',
 'Everyone who wanted to get up but inquired did know that, because the Auto-Navi being instead of a bypassed road attached a green fields.',
 'Jeder kennt dieses Kündigt er schon mal voll auf der Auto-Navi statt einer Durchhungsstrasse eine Grünweche anweist. ().',
 'Our German bus operator is here. You will see the East European telescopic results of the appropriate sheikse classification.',
 'Everybody knows that, who had to prove their own aging because the Auto-Navi showed a green meadow rather than a bypass road.',
 'You know, that was a thing of the past, because Auto-navigation was fast and had no other way around it and a green grass.',
 'The others day at 11.00 pm done it really "misleadingly for" (mid-October). It was just a few minutes from the 3.00 pmpm!',
 'Everyone needs to get up in courage, because it was a greenfield instead of a road bypass.',
 'Hundreds o

In [40]:
mbr = MBR(chrf, cached_sampler)  # This version of MBR is what Eikema and Aziz (2021) call MBR N-by-S

In [41]:
y_mbr = mbr.decode(example_x, hyp_space)
y_mbr

'Everyone who need help is aware of that because the Auto-Navi displayed a green meadow instead of a bypass road.'

In [42]:
chrf(example_x, y_mbr, example_y)

0.4252269767015976

In [43]:
mbr.mu(example_x, y_mbr)

0.3950191490559801

In [44]:
for h, mu in sorted(zip(hyp_space, mbr.mus(example_x, hyp_space)), key=lambda pair: pair[1], reverse=True):
    print(f"{mu:.4f}\t{h}")

0.3994	Something everybody knows that Chiclana dazzled... because the Auto-Navi displayed a green meadow instead of a bypass.
0.3821	This is something everybody knows, that has already had to orient themselves on because the AutoNavi displayed a green meadow instead of a bypass road.
0.3793	This seems to be something that everybody knew, who ever had to get up scooter, because the Auto-Navi displayed a green grass instead of a bypass.
0.3784	You know this for everybody who has had to make a start because the Auto @-@Navi showed a green meadow instead of a bypass.
0.3666	Everyone who need help is aware of that because the Auto-Navi displayed a green meadow instead of a bypass road.
0.3654	That was noticed by all who had to get excivated because the Auto-Navi showed a green meadow instead of a by-pass road.
0.3585	Everyone who had to get excited knew that because Auto-Navi displayed a greenful grass instead of bypass roads.
0.3570	Everyone who has had to get used to this has been aware o